# Solving the Poisson equation
follows this tutorial: https://jsdokken.com/dolfinx-tutorial/chapter1/fundamentals_code.html

In [1]:
import dolfinx
print(f"DOLFINx version: {dolfinx.__version__} based on GIT commit: {dolfinx.git_commit_hash} of https://github.com/FEniCS/dolfinx/")

DOLFINx version: 0.7.2 based on GIT commit: 4604094c8c3e6dbf6034b25df5ee3ccd1ec2d3b3 of https://github.com/FEniCS/dolfinx/


In [2]:
from mpi4py import MPI
from dolfinx import mesh
# Unit square is [0,1] x [0,1]
# MPI argument indicates how to parallelise computation
# this will split the mesh into 8x8
# Quadralateral cells are 4 sisded
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)

In [3]:
from dolfinx.fem import FunctionSpace
# 1st argument is the mesh to act on
# 2nd argument specifies the 'family' of the element 
#3 rd argument is the degree (i.e. 2d for a vector 1d for pressure)
V = FunctionSpace(domain, ("Lagrange", 1))

from dolfinx import fem
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

In [4]:
import numpy
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

In [5]:
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

In [6]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [7]:
from dolfinx import default_scalar_type
f = fem.Constant(domain, default_scalar_type(-6))

In [8]:
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

In [9]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [10]:
V2 = fem.FunctionSpace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

In [11]:
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

In [12]:
error_max = numpy.max(numpy.abs(uD.x.array-uh.x.array))
# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

Error_L2 : 8.24e-03
Error_max : 2.00e-15


In [13]:
import pyvista
pyvista.set_jupyter_backend("client")
print(pyvista.global_theme.jupyter_backend)


client


In [14]:
from dolfinx import plot
pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

In [15]:
plotter = pyvista.Plotter(notebook=True)
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:39385/index.html?ui=P_0x7fab3f9ce260_0&reconnect=auto" class="pyvi…

In [16]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

In [17]:
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:39385/index.html?ui=P_0x7fab299735b0_1&reconnect=auto" class="pyvi…

 JS Error => Malformed message:  {"wslink": "1.0", "id": "publish:trame.vtk.delta:0", "result": {"parent": "0x0", "id": 2, "type": "vtkXOpenGLRenderWindow", "properties": {"numberOfLayers": 2}, "dependencies": [{"parent": "00005597160d3e00", "id": "00005597160d0d10", "type": "vtkOpenGLRenderer", "properties": {"background": [1.0, 1.0, 1.0], "background2": [0.2, 0.2, 0.2], "viewport": [0.0, 0.0, 1.0, 1.0], "twoSidedLighting": 1, "lightFollowCamera": 1, "layer": 0, "preserveColorBuffer": 0, "preserveDepthBuffer": 0, "nearClippingPlaneTolerance": 0.001, "clippingRangeExpansion": 0.5, "useShadows": 0, "useDepthPeeling": 0, "occlusionRatio": 0.0, "maximumNumberOfPeels": 4, "interactive": 1}, "dependencies": [{"parent": "00005597160d0d10", "id": "00005597160b4a00", "type": "vtkOpenGLCamera", "properties": {"focalPoint": [0.5000000000000001, 0.5000000000000001, 0.0], "position": [0.5000000000000001, 0.5000000000000001, 2.732050807568878], "viewUp": [0.0, 1.0, 0.0], "clippingRange": [2.5590521

In [18]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

Widget(value='<iframe src="http://localhost:35147/index.html?ui=P_0x7f2060197c70_2&reconnect=auto" class="pyvi…

 JS Error => Malformed message:  {"wslink": "1.0", "id": "publish:trame.vtk.delta:0", "result": {"parent": "0x0", "id": 3, "type": "vtkXOpenGLRenderWindow", "properties": {"numberOfLayers": 2}, "dependencies": [{"parent": "000055f5b8ca9ea0", "id": "000055f5b8c99ce0", "type": "vtkOpenGLRenderer", "properties": {"background": [1.0, 1.0, 1.0], "background2": [0.2, 0.2, 0.2], "viewport": [0.0, 0.0, 1.0, 1.0], "twoSidedLighting": 1, "lightFollowCamera": 1, "layer": 0, "preserveColorBuffer": 0, "preserveDepthBuffer": 0, "nearClippingPlaneTolerance": 0.001, "clippingRangeExpansion": 0.5, "useShadows": 0, "useDepthPeeling": 0, "occlusionRatio": 0.0, "maximumNumberOfPeels": 4, "interactive": 1}, "dependencies": [{"parent": "000055f5b8c99ce0", "id": "000055f5b8b70a40", "type": "vtkOpenGLCamera", "properties": {"focalPoint": [0.5, 0.5, 2.5], "position": [4.199214280684376, 4.199214280684376, 6.199214280684376], "viewUp": [0.0, 0.0, 1.0], "clippingRange": [3.470837222189011, 10.118991549263171]}},